# Example Recurrent Neural Network in tflearn

Largely taken from:

https://github.com/tflearn/tflearn/blob/master/examples/nlp/lstm_generator_cityname.py
https://github.com/tflearn/tflearn/blob/master/examples/nlp/lstm_generator_shakespeare.py

A really nice introduction: http://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [43]:
import numpy as np
import matplotlib as plt
import tflearn 

import os
from six import moves
import ssl
from tflearn.data_utils import *
import tensorflow as tf

In [86]:
char_lim = 70
title_file = 'titles_%d_char_lim.txt' %char_lim

In [87]:
fl = open(title_file, 'r')
tls = fl.readlines()
fl.close()
max_len = 0
for t in tls[:200]:
    if len(t) > max_len:
        max_len = len(t)
#     print(t)
print(len(tls))
print(max_len)

20779
70


Convert example data

In [88]:
#path = "US_Cities.txt"
#path = "titles.txt"
path = title_file

maxlen = char_lim

string_utf8 = open(path, "r").read()
X, Y, char_idx = \
    textfile_to_semi_redundant_sequences(path, seq_maxlen=maxlen, redun_step=3)

Vectorizing text...
Text total length: 1,033,082
Distinct chars   : 93
Total sequences  : 344,338


In [89]:
tf.reset_default_graph()

In [90]:
batch_size = 256

g = tflearn.input_data(shape=[None, maxlen, len(char_idx)])
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512)
g = tflearn.dropout(g, 0.5)
g = tflearn.fully_connected(g, len(char_idx), activation='softmax')
g = tflearn.regression(g, optimizer='adam', loss='categorical_crossentropy',
                       learning_rate=0.001)

m = tflearn.SequenceGenerator(g, dictionary=char_idx,
                              seq_maxlen=maxlen,
                              clip_gradients=5.0,
                              checkpoint_path='titles_%d' %batch_size)

In [64]:
#m.load('rnn_batch_size_128')

In [91]:
seed = random_sequence_from_string(string_utf8, maxlen)
m.fit(X, Y, validation_set=0.1, batch_size=batch_size,
      n_epoch=3, run_id='titles_%d' %batch_size)
print("-- TESTING...")
print("-- Test with temperature of 1.2 --")
print(m.generate(30, temperature=1.2, seq_seed=seed).encode('utf-8'))
print("-- Test with temperature of 1.0 --")
print(m.generate(30, temperature=1.0, seq_seed=seed).encode('utf-8'))
print("-- Test with temperature of 0.5 --")
print(m.generate(30, temperature=0.5, seq_seed=seed).encode('utf-8'))

Training Step: 3632  | total loss: 1.52675 | time: 11196.233s
| Adam | epoch: 003 | loss: 1.52675 -- iter: 309760/309904
Training Step: 3633  | total loss: 1.53117 | time: 11722.556s
| Adam | epoch: 003 | loss: 1.53117 | val_loss: 1.45796 -- iter: 309904/309904
--
INFO:tensorflow:/home/michelle/Dropbox/Project/JEDI_2017/titles_256-3633 is not in all_model_checkpoint_paths. Manually adding it.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
-- TESTING...
-- Test with temperature of 1.2 --
b'ranes in AdS3 spacetime\nOn Type IIA String Theory on the PP-wave Backgers equetric for ondtrian Conr'
-- Test with temperature of 1.0 --
b'ranes in AdS3 spacetime\nOn Type IIA String Theory on the PP-wave Backgrounds Micres vax D5ent Field '
-- Test with temperature of 0.5 --
b'ranes in AdS3 spaceti

In [92]:
m.save('rnn_v2_batch_size_256')

INFO:tensorflow:/home/michelle/Dropbox/Project/JEDI_2017/rnn_v2_batch_size_256 is not in all_model_checkpoint_paths. Manually adding it.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
